# Install package ที่จำเป็นต้องใช้

In [4]:
!pip install selenium #selenium
!pip install bs4 #beautiful soup

ERROR: Invalid requirement: '#selenium'
ERROR: Invalid requirement: '#beautiful'


# Import function ที่จำเป็น

In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
import bs4
import time
import re
import csv 
from bs4 import BeautifulSoup 
from msedge.selenium_tools import Edge,EdgeOptions

# สร้าง Web driver ของ Firefox
โดยการเช็ค Version Driver ก่อน Download https://github.com/mozilla/geckodriver/releases

เพื่อใช้ในการเปิด Website ที่ต้องการ Scraping ( Selenium เหมาะกับการใช้เปิด website ประเภท Dynamic Website )

ในส่วน options.binary_location ให้เลือกที่อยู่ของ Firefox driver = webdriver.Firefox executable_path ให้ทำการเก็บไฟล์ที่ได้จากการ download driver บนเว็ปไซต์ไว้ในตำแหน่งเดียวกันกับตัวไฟล์

In [6]:
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(executable_path=r'C:\Users\G15\Downloads\shoppee\geckodriver.exe', options=options)
driver.get('https://shopee.co.th/search?keyword=c-vitt%20%E0%B9%80%E0%B8%A2%E0%B8%A5%E0%B8%A5%E0%B8%B5%E0%B9%88')

WebDriverException: Message: 'geckodriver.exe' executable needs to be in PATH. 


เรียกใช้ Selenium และเปิดเว็ปที่ต้องการ scrap ซึ่ง website ที่ใช้ Shopee จะมีหน้าในการเลือกภาษาและ Zoomout page เพื่อโหลดข้อมูล 
( ขั้นตอนนี้ขึ้นอยู่แต่ละ website ที่จะนำไป scrap ) 

In [ ]:
#เลือกภาษาไทย Selenium
thai_button = driver.find_element("xpath",'/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button').click()
time.sleep(6)

In [ ]:
#Zoom out เพื่อให้ load ทั่วหน้าจอ
driver.execute_script("document.body.style.MozTransform='scale(0.1)';")
driver.execute_script('document.body.style.MozTransformOrigin = "0 0";')

# เริ่มต้นขั้นตอน Scraping โดยใช้ BS4

In [ ]:
data = driver.page_source #ดึงข้อมูลจากหน้าเว็บ
soup = bs4.BeautifulSoup(data) #จัดในรูปแบบ BeautifulSoup
time.sleep(6)

In [ ]:
soup.select(".row>div") #css selector

In [ ]:
el=soup.select(".row>div")

In [ ]:
len(el)

ทดลอง Scrap ชื่อสินค้า

In [ ]:
e = el[0] #แทนกล่องของสินค้าแต่ละกล่อง

In [ ]:
#ดึงข้อมูลชื่อ
e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)").text.strip()

ทอลอง Scrap ราคา

In [ ]:
#ราคา A baht - B baht, ราคาเต็ม
e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2)").text.strip()

In [ ]:
#รวม ราคาถูกสุด
try:
    p = e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2)").text.strip()
except:
    p = e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2)").text.strip()
p = p.replace(",", "").replace("฿", "").replace(" ", "")
try:
    start_price = float(p)
except:
    price = p.rpartition('-')[0]
    start_price = float(price)

start_price

ทดลอง Scrap ยอดขาย


In [ ]:
#Scrape ยอดขาย
sales = e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3) > div:nth-child(2)").text.strip().replace("พัน","000").replace("ล้าน","000000")
try:
    sales = float(re.sub('\D','',sales))
except:
    sales = 0
sales

ทดลอง Scrap จังหวัด

In [ ]:
#Scrape จังหวัด
#จังหวัด
e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(4)").text.strip()

## นำขั้นตอนการเก็บข้อมูลทั้งหมดมาเข้า Loop เพื่อทำการ Scrap สินค้าทั้งหมดในทุกหน้า

In [ ]:
time.sleep(6)
records=[]
for i in range(2):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    el=soup.select(".row>div")
    for e in el:
        name = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text.strip()
        try:
            p = e.select_one("div.col-xs-2-4 > a > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2)").text.strip()
            if p == '':
                print(1/0)
        except:
            p = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()
        p = p.replace(",", "").replace("฿", "").replace(" ", "")
        try:
            start_price = float(p)
        except:
            price = p.rpartition('-')[0]
            start_price = float(price)

        sales = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3)").text.strip().replace("พัน","00").replace("ล้าน","00000")
        try:
            sales = float(re.sub('\D','',sales))
        except:
            sales = 0
        province = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(4)").text.strip()
        #print(name, p)
        records.append([name, start_price,sales,province])

In [ ]:
len(records)

## นำข้อมูลที่ได้ใส่ลงใน DataFrame

In [ ]:
#ทำเป็น DataFrame
df = pd.DataFrame(records, columns=['itemname','price range','sales','province'])
df

แปลงเป็นไฟล์ Excel บันทึกอักษรเป็นแบบ มาตรฐาน UTF-8

In [ ]:
df.to_excel('Shopee_C-vitt.xlsx', encoding='utf-8')